In [200]:
import urllib.request
import http.cookiejar
import urllib
from bs4 import BeautifulSoup
import requests
import random
import re

In [62]:
USER_AGENT_LIST = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
    "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]

In [338]:
def get_response(url,params):
    headers = {'user-agent':random.choice(USER_AGENT_LIST)}

    resp = requests.get(url,
                        headers=headers,
                        params=params,
                        timeout=3)
    resp.encoding = "UTF-8"
    resp.raise_for_status()
    soup = BeautifulSoup(resp.content, 'lxml')
    return soup

def scholar_kitchen(soup_unit):
    Ad = soup_unit.select_one('div.gs_a').text.replace(u'\xa0', u' ')
    Tails = soup_unit.select('div.gs_fl > a')
    TailsT = [x.text for x in Tails]
    KP_CD = [i for i,j in enumerate(TailsT) if re.match('^Cited*',j) is not None]
    KP_url = soup_unit.select_one('h3.gs_rt > a[href]')
    KP_pdfurl = soup_unit.select_one('div.gs_or_ggsm > a[href]')
    KP_desc = soup_unit.select_one('div.gs_rs')
    
    Title = soup_unit.select_one('h3.gs_rt').text
    FAuthor = re.split(r',| - ',Ad)[0]
    Descript = KP_desc.text.replace(u'\xa0', u' ') if KP_desc is not None else ''
    Cite_count =  Tails[KP_CD[0]].text.split('Cited by ')[1] if len(KP_CD) is not 0 else ''
    Url = KP_url.attrs['href'] if KP_url is not None else ''
    PDF_url = KP_pdfurl.attrs['href'] if KP_pdfurl is not None else ''

    
    Result = {'Title':Title,
             'FAuthor':FAuthor,
             'Url':Url,
             'PDFurl':PDF_url,
             'Description':Descript,
             'Cite_count':Cite_count}
    return Result

In [114]:
httpbin='https://httpbin.org'
google='https://google.com'
twyahoo='https://tw.yahoo.com'
gscholar='https://scholar.google.com.tw/scholar'

In [184]:
params = {'start':'0',
          'q':'dota2',
          'hl':'en',
          'as_sdt':'0,5',
          'as_ylo':'2016',
          'as_yhi':'2016'}

In [339]:
meal = get_response(gscholar,params)
soups = meal.select('div.gs_r.gs_or.gs_scl')
ingreds = []
for i, soup in enumerate(soups):
    try:
        ingreds.append(scholar_kitchen(soup))
    except Exception as e: 
        print(i)
        print(e)
        continue
    
    

In [340]:
ingreds

[{'Title': 'Fuzzy controller based AI for dynamic difficulty adjustment for defense of the Ancient 2 (DotA2)',
  'FAuthor': 'NPH Pratama',
  'Url': 'https://ieeexplore.ieee.org/abstract/document/7828640/',
  'PDFurl': '',
  'Description': "Defense of the Ancient 2 (DotA2) is a game with a huge player base achieving almost 11 millions players during the time when this paper was written. Even though this game is more focused on its multiplayer players versus players mode, it's bot match mode is still a viable …",
  'Cite_count': '3'},
 {'Title': "[CITATION][C] Virtual Reality Takes Stage At World's Largest Esports Tournament, The International For Dota2",
  'FAuthor': 'W Green',
  'Url': '',
  'PDFurl': '',
  'Description': '',
  'Cite_count': '2'},
 {'Title': 'VR UR< 3 Virtual Reality Impact on> Libraries',
  'FAuthor': 'C Bryant',
  'Url': 'https://digitalcommons.wku.edu/kcc/2016_Conference/Day_1/30/',
  'PDFurl': '',
  'Description': '… Louis, and Southwestern University offer scholar